In [1]:
#import dependancies
import pandas as pd
from sqlalchemy import create_engine

In [2]:
#connect to database
rds_connection_string = "postgres:root123@localhost:5432/NBA_DB"
engine = create_engine(f'postgresql://{rds_connection_string}')
engine.table_names()

['players',
 'seasons_stats',
 'team',
 'team_player',
 'player',
 'player_stats',
 'team_stats',
 'salary',
 'player_stat']

In [3]:
#loading file
csv = "Resources/salary.csv"

salary_df = pd.read_csv(csv)
salary_df.head()

,Register Value,Player Name,Salary,Season Start,Season End,Team,Full Team Name
0,1,A.C. Green,1750000,1990,1991,LAL,Los Angeles Lakers
1,354,A.C. Green,1750000,1991,1992,LAL,Los Angeles Lakers
2,727,A.C. Green,1750000,1992,1993,LAL,Los Angeles Lakers
3,1103,A.C. Green,1885000,1993,1994,PHO,Phoenix Suns
4,1497,A.C. Green,6472600,1994,1995,PHO,Phoenix Suns


In [4]:
#dropping the column that is not required
salary_df = salary_df.drop(columns=['Full Team Name'], axis=1)

In [5]:
# rename the columns based on other tables
salary_df = salary_df.rename(columns={"Register Value": "salary_id",
                                            "Player Name": "full_name",
                                            "Salary": "salary",
                                            "Season Start": "start_season",
                                            "Season End": "end_season",
                                            "Team": "id_team"}).dropna(how='all')
salary_df.head()

,salary_id,full_name,salary,start_season,end_season,id_team
0,1,A.C. Green,1750000,1990,1991,LAL
1,354,A.C. Green,1750000,1991,1992,LAL
2,727,A.C. Green,1750000,1992,1993,LAL
3,1103,A.C. Green,1885000,1993,1994,PHO
4,1497,A.C. Green,6472600,1994,1995,PHO


In [7]:
#bring in the SQL table
player_table = pd.read_sql_query("SELECT player_id, full_name FROM player where is_active = true", con=engine)
player_table.head()

,player_id,full_name
0,203500,Steven Adams
1,1628389,Bam Adebayo
2,200746,LaMarcus Aldridge
3,1629638,Nickeil Alexander-Walker
4,1628960,Grayson Allen


In [8]:
#Merge the tables together to get the player id
result = pd.merge(player_table, salary_df, how='inner', on=["full_name"])
result.head()

,player_id,full_name,salary_id,salary,start_season,end_season,id_team
0,203500,Steven Adams,9581,2090880,2013,2014,OKC
1,203500,Steven Adams,10096,2184960,2014,2015,OKC
2,203500,Steven Adams,10652,2279040,2015,2016,OKC
3,203500,Steven Adams,11257,3140517,2016,2017,OKC
4,203500,Steven Adams,11773,22471910,2017,2018,OKC


In [10]:
#Final Salary to drop the full name column
final_salary = result.drop(columns=["full_name"])
final_salary.head()

,player_id,salary_id,salary,start_season,end_season,id_team
0,203500,9581,2090880,2013,2014,OKC
1,203500,10096,2184960,2014,2015,OKC
2,203500,10652,2279040,2015,2016,OKC
3,203500,11257,3140517,2016,2017,OKC
4,203500,11773,22471910,2017,2018,OKC


In [13]:
final_salary.to_sql(name='salary', con=engine, if_exists='append', index=False)